In [107]:
import pandas as pd
import numpy as np
from collections import Counter as c
from sklearn.preprocessing import LabelEncoder


from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# from lazypredict.Supervised import LazyClassifier

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor # why?
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor


from xgboost import XGBRegressor , XGBRFRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor


In [59]:
train = pd.read_csv("train_0OECtn8.csv")
test = pd.read_csv("test_1zqHu22.csv")
sample = pd.read_csv("sample_submission_JPlpRcN.csv")

train.shape, test.shape, sample.shape

((89197, 10), (11121, 9), (11121, 2))

In [60]:
sample.head()

,row_id,engagement_score
0,89198,5.0
1,89199,5.0
2,89200,5.0
3,89201,5.0
4,89202,5.0


In [63]:
c(sample["engagement_score"])

Counter({5.0: 11121})

In [65]:
# test.head()

In [66]:
# c(test["user_id"])

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89197 entries, 0 to 89196
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_id            89197 non-null  int64  
 1   user_id           89197 non-null  int64  
 2   category_id       89197 non-null  int64  
 3   video_id          89197 non-null  int64  
 4   age               89197 non-null  int64  
 5   gender            89197 non-null  object 
 6   profession        89197 non-null  object 
 7   followers         89197 non-null  int64  
 8   views             89197 non-null  int64  
 9   engagement_score  89197 non-null  float64
dtypes: float64(1), int64(7), object(2)
memory usage: 6.8+ MB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11121 entries, 0 to 11120
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   row_id       11121 non-null  int64 
 1   user_id      11121 non-null  int64 
 2   category_id  11121 non-null  int64 
 3   video_id     11121 non-null  int64 
 4   age          11121 non-null  int64 
 5   gender       11121 non-null  object
 6   profession   11121 non-null  object
 7   followers    11121 non-null  int64 
 8   views        11121 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 782.1+ KB


In [6]:
train.head()

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score
0,1,19990,37,128,24,Male,Student,180,1000,4.33
1,2,5304,32,132,14,Female,Student,330,714,1.79
2,3,1840,12,24,19,Male,Student,180,138,4.35
3,4,12597,23,112,19,Male,Student,220,613,3.77
4,5,13626,23,112,27,Male,Working Professional,220,613,3.13


In [8]:
# c(train["category_id"])

In [12]:
# c(train["video_id"])

In [13]:
c(train["gender"])

Counter({'Male': 52397, 'Female': 36800})

In [14]:
c(train["profession"])

Counter({'Student': 44638, 'Working Professional': 17719, 'Other': 26840})

In [15]:
df = train.copy()

In [16]:
test.shape

(11121, 9)

In [50]:
test_col = test.columns.to_list()

train_col = ['row_id',
 'user_id',
 'category_id',
 'video_id',
 'age',
 'gender',
 'profession',
 'followers',
 'views',]
# test_col

In [49]:
df = train.merge(test,how='outer',left_on= train_col,   right_on=test_col)
# df

In [41]:
test.shape, train.shape, len(test) + len(train)

((11121, 9), (89197, 10), 100318)

In [38]:
len(test) + len(train)

100318

In [46]:
# df.iloc[2:5, 4:8]

In [45]:
df.head()

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score
0,1,19990,37,128,24,Male,Student,180,1000,4.33
1,2,5304,32,132,14,Female,Student,330,714,1.79
2,3,1840,12,24,19,Male,Student,180,138,4.35
3,4,12597,23,112,19,Male,Student,220,613,3.77
4,5,13626,23,112,27,Male,Working Professional,220,613,3.13


In [51]:
df

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score
0,1,19990,37,128,24,Male,Student,180,1000,4.33
1,2,5304,32,132,14,Female,Student,330,714,1.79
2,3,1840,12,24,19,Male,Student,180,138,4.35
3,4,12597,23,112,19,Male,Student,220,613,3.77
4,5,13626,23,112,27,Male,Working Professional,220,613,3.13
...,...,...,...,...,...,...,...,...,...,...
100313,100314,26336,25,140,21,Male,Student,240,317,NaN
100314,100315,6772,8,100,19,Female,Student,280,628,NaN
100315,100316,2042,16,98,22,Male,Student,270,462,NaN
100316,100317,24626,8,16,33,Male,Other,280,628,NaN


In [55]:
#Label encoding
def convert_to_numerical_label_encoding(dataset):
    enc = LabelEncoder()
    for i in dataset.columns:
        if(dataset[i].dtype == "object"):
            dataset[i] = enc.fit_transform(dataset[i])
            
    return dataset


In [56]:
df1 = convert_to_numerical_label_encoding(df.copy())
df1

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score
0,1,19990,37,128,24,1,1,180,1000,4.33
1,2,5304,32,132,14,0,1,330,714,1.79
2,3,1840,12,24,19,1,1,180,138,4.35
3,4,12597,23,112,19,1,1,220,613,3.77
4,5,13626,23,112,27,1,2,220,613,3.13
...,...,...,...,...,...,...,...,...,...,...
100313,100314,26336,25,140,21,1,1,240,317,NaN
100314,100315,6772,8,100,19,0,1,280,628,NaN
100315,100316,2042,16,98,22,1,1,270,462,NaN
100316,100317,24626,8,16,33,1,0,280,628,NaN


In [57]:
print(c(df["profession"]))
print(c(df1["profession"]))

Counter({'Student': 50210, 'Other': 30206, 'Working Professional': 19902})
Counter({1: 50210, 0: 30206, 2: 19902})


In [68]:
df2 = df1.drop(columns=["row_id", "user_id"])
df2.shape, df1.shape

((100318, 8), (100318, 10))

In [78]:
print(train.shape, test.shape)

df_train = df2[df2["engagement_score"].isnull() == False]
df_test = df2[df2["engagement_score"].isnull() == True]
print("*"*100)

print(df_train.shape, df_test.shape)

(89197, 10) (11121, 9)
****************************************************************************************************
(89197, 8) (11121, 8)


In [97]:
print(df_test.shape)
df_test = df_test.drop(columns=["engagement_score"])
print(df_test.shape)

(11121, 8)
(11121, 7)


In [80]:
X = df_train.iloc[:,:-1]
y = df_train.iloc[:,-1]

In [81]:
# df_train.head(1)

In [88]:
m = XGBRFRegressor()
m.fit(X,y)

XGBRFRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bytree=1, enable_categorical=False, gamma=0, gpu_id=-1,
               importance_type=None, interaction_constraints='',
               max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=100, n_jobs=4,
               num_parallel_tree=100, objective='reg:squarederror',
               predictor='auto', random_state=0, reg_alpha=0,
               scale_pos_weight=1, tree_method='exact', validate_parameters=1,
               verbosity=None)

In [98]:
# p = m.predict(df_test)

In [99]:
# X.shape, df_test.shape

In [103]:
def model_building(model, x_feature, y_target, x_test):
    m = model
    m.fit(x_feature, y_target)
    pred = m.predict(x_test)
    return pred
    

def create_submission_file(sample_submission, y_pred_final, file_name):
    
    sample_submission["engagement_score"] = y_pred_final
    sample_submission.to_csv(file_name+'.csv', index=False)
    
    return file_name + " csv submission created successful"

In [105]:
prde_y = model_building(XGBRFRegressor(), X, y, df_test)
create_submission_file(sample.copy(), prde_y, "XGBR_1")

'XGBR_1 csv submission created successful'

In [108]:
prde_y = model_building(LGBMRegressor(), X, y, df_test)
create_submission_file(sample.copy(), prde_y, "LGBR_1")

'LGBR_1 csv submission created successful'

In [109]:
prde_y = model_building(CatBoostRegressor(), X, y, df_test)
create_submission_file(sample.copy(), prde_y, "CatBoost_1")

Learning rate set to 0.083241
0:	learn: 0.8436150	total: 114ms	remaining: 1m 53s
1:	learn: 0.8262294	total: 144ms	remaining: 1m 11s
2:	learn: 0.8111629	total: 194ms	remaining: 1m 4s
3:	learn: 0.7984846	total: 210ms	remaining: 52.4s
4:	learn: 0.7870495	total: 220ms	remaining: 43.8s
5:	learn: 0.7776366	total: 228ms	remaining: 37.7s
6:	learn: 0.7696841	total: 235ms	remaining: 33.3s
7:	learn: 0.7621992	total: 241ms	remaining: 29.9s
8:	learn: 0.7560388	total: 247ms	remaining: 27.2s
9:	learn: 0.7508945	total: 253ms	remaining: 25.1s
10:	learn: 0.7461345	total: 260ms	remaining: 23.3s
11:	learn: 0.7421002	total: 266ms	remaining: 21.9s
12:	learn: 0.7380994	total: 273ms	remaining: 20.7s
13:	learn: 0.7349376	total: 279ms	remaining: 19.6s
14:	learn: 0.7325848	total: 287ms	remaining: 18.8s
15:	learn: 0.7304653	total: 293ms	remaining: 18s
16:	learn: 0.7282534	total: 299ms	remaining: 17.3s
17:	learn: 0.7262978	total: 305ms	remaining: 16.7s
18:	learn: 0.7240151	total: 312ms	remaining: 16.1s
19:	learn: 

162:	learn: 0.6901637	total: 1.35s	remaining: 6.95s
163:	learn: 0.6901347	total: 1.36s	remaining: 6.93s
164:	learn: 0.6901075	total: 1.37s	remaining: 6.92s
165:	learn: 0.6900655	total: 1.37s	remaining: 6.9s
166:	learn: 0.6900039	total: 1.38s	remaining: 6.89s
167:	learn: 0.6899626	total: 1.39s	remaining: 6.88s
168:	learn: 0.6899287	total: 1.4s	remaining: 6.86s
169:	learn: 0.6898915	total: 1.4s	remaining: 6.84s
170:	learn: 0.6898650	total: 1.41s	remaining: 6.82s
171:	learn: 0.6898076	total: 1.41s	remaining: 6.8s
172:	learn: 0.6897684	total: 1.42s	remaining: 6.78s
173:	learn: 0.6897053	total: 1.43s	remaining: 6.77s
174:	learn: 0.6896731	total: 1.43s	remaining: 6.75s
175:	learn: 0.6896437	total: 1.44s	remaining: 6.73s
176:	learn: 0.6895926	total: 1.44s	remaining: 6.71s
177:	learn: 0.6895785	total: 1.45s	remaining: 6.69s
178:	learn: 0.6895484	total: 1.46s	remaining: 6.68s
179:	learn: 0.6895109	total: 1.46s	remaining: 6.66s
180:	learn: 0.6894517	total: 1.47s	remaining: 6.65s
181:	learn: 0.68

330:	learn: 0.6850116	total: 2.56s	remaining: 5.17s
331:	learn: 0.6849880	total: 2.57s	remaining: 5.16s
332:	learn: 0.6849685	total: 2.57s	remaining: 5.16s
333:	learn: 0.6849487	total: 2.58s	remaining: 5.15s
334:	learn: 0.6849283	total: 2.59s	remaining: 5.13s
335:	learn: 0.6849196	total: 2.59s	remaining: 5.13s
336:	learn: 0.6848888	total: 2.6s	remaining: 5.12s
337:	learn: 0.6848709	total: 2.61s	remaining: 5.11s
338:	learn: 0.6848445	total: 2.63s	remaining: 5.13s
339:	learn: 0.6848338	total: 2.65s	remaining: 5.14s
340:	learn: 0.6848251	total: 2.66s	remaining: 5.14s
341:	learn: 0.6848015	total: 2.67s	remaining: 5.13s
342:	learn: 0.6847829	total: 2.67s	remaining: 5.12s
343:	learn: 0.6847660	total: 2.68s	remaining: 5.11s
344:	learn: 0.6847448	total: 2.69s	remaining: 5.1s
345:	learn: 0.6847138	total: 2.69s	remaining: 5.09s
346:	learn: 0.6847033	total: 2.7s	remaining: 5.08s
347:	learn: 0.6846937	total: 2.7s	remaining: 5.06s
348:	learn: 0.6846790	total: 2.71s	remaining: 5.05s
349:	learn: 0.68

489:	learn: 0.6822193	total: 3.77s	remaining: 3.92s
490:	learn: 0.6822026	total: 3.77s	remaining: 3.91s
491:	learn: 0.6821888	total: 3.79s	remaining: 3.91s
492:	learn: 0.6821801	total: 3.79s	remaining: 3.9s
493:	learn: 0.6821718	total: 3.8s	remaining: 3.89s
494:	learn: 0.6821506	total: 3.81s	remaining: 3.88s
495:	learn: 0.6821410	total: 3.82s	remaining: 3.88s
496:	learn: 0.6821194	total: 3.82s	remaining: 3.87s
497:	learn: 0.6821127	total: 3.83s	remaining: 3.86s
498:	learn: 0.6821059	total: 3.84s	remaining: 3.85s
499:	learn: 0.6820907	total: 3.85s	remaining: 3.85s
500:	learn: 0.6820737	total: 3.85s	remaining: 3.84s
501:	learn: 0.6820591	total: 3.87s	remaining: 3.83s
502:	learn: 0.6820512	total: 3.87s	remaining: 3.83s
503:	learn: 0.6820346	total: 3.88s	remaining: 3.82s
504:	learn: 0.6820103	total: 3.88s	remaining: 3.81s
505:	learn: 0.6819908	total: 3.89s	remaining: 3.8s
506:	learn: 0.6819726	total: 3.9s	remaining: 3.79s
507:	learn: 0.6819579	total: 3.9s	remaining: 3.78s
508:	learn: 0.681

676:	learn: 0.6794911	total: 5.18s	remaining: 2.47s
677:	learn: 0.6794805	total: 5.19s	remaining: 2.46s
678:	learn: 0.6794680	total: 5.2s	remaining: 2.46s
679:	learn: 0.6794558	total: 5.21s	remaining: 2.45s
680:	learn: 0.6794405	total: 5.22s	remaining: 2.44s
681:	learn: 0.6794170	total: 5.23s	remaining: 2.44s
682:	learn: 0.6794088	total: 5.24s	remaining: 2.43s
683:	learn: 0.6793964	total: 5.25s	remaining: 2.42s
684:	learn: 0.6793862	total: 5.26s	remaining: 2.42s
685:	learn: 0.6793627	total: 5.27s	remaining: 2.41s
686:	learn: 0.6793536	total: 5.28s	remaining: 2.4s
687:	learn: 0.6793453	total: 5.28s	remaining: 2.4s
688:	learn: 0.6793393	total: 5.29s	remaining: 2.39s
689:	learn: 0.6793274	total: 5.3s	remaining: 2.38s
690:	learn: 0.6793096	total: 5.31s	remaining: 2.37s
691:	learn: 0.6793039	total: 5.32s	remaining: 2.37s
692:	learn: 0.6792903	total: 5.33s	remaining: 2.36s
693:	learn: 0.6792808	total: 5.33s	remaining: 2.35s
694:	learn: 0.6792727	total: 5.34s	remaining: 2.34s
695:	learn: 0.67

836:	learn: 0.6776768	total: 6.4s	remaining: 1.25s
837:	learn: 0.6776578	total: 6.41s	remaining: 1.24s
838:	learn: 0.6776524	total: 6.42s	remaining: 1.23s
839:	learn: 0.6776430	total: 6.42s	remaining: 1.22s
840:	learn: 0.6776333	total: 6.43s	remaining: 1.22s
841:	learn: 0.6776184	total: 6.44s	remaining: 1.21s
842:	learn: 0.6776039	total: 6.44s	remaining: 1.2s
843:	learn: 0.6775937	total: 6.45s	remaining: 1.19s
844:	learn: 0.6775904	total: 6.46s	remaining: 1.18s
845:	learn: 0.6775805	total: 6.46s	remaining: 1.18s
846:	learn: 0.6775734	total: 6.47s	remaining: 1.17s
847:	learn: 0.6775589	total: 6.48s	remaining: 1.16s
848:	learn: 0.6775507	total: 6.48s	remaining: 1.15s
849:	learn: 0.6775400	total: 6.49s	remaining: 1.15s
850:	learn: 0.6775313	total: 6.5s	remaining: 1.14s
851:	learn: 0.6775152	total: 6.5s	remaining: 1.13s
852:	learn: 0.6775102	total: 6.51s	remaining: 1.12s
853:	learn: 0.6774994	total: 6.51s	remaining: 1.11s
854:	learn: 0.6774900	total: 6.52s	remaining: 1.1s
855:	learn: 0.677

'CatBoost_1 csv submission created successful'

In [110]:
prde_y = model_building(LinearRegression(), X, y, df_test)
create_submission_file(sample.copy(), prde_y, "LinearRegression_1")

'LinearRegression_1 csv submission created successful'

In [111]:
prde_y = model_building(DecisionTreeRegressor(), X, y, df_test)
create_submission_file(sample.copy(), prde_y, "DecisionTreeRegressor_1")

'DecisionTreeRegressor_1 csv submission created successful'

In [112]:
prde_y = model_building(RandomForestRegressor(), X, y, df_test)
create_submission_file(sample.copy(), prde_y, "RandomForestRegressor_1")

'RandomForestRegressor_1 csv submission created successful'

In [115]:
# !pip install tensorflow_docs

In [118]:

#OR

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.preprocessing import sequence


from tensorflow.keras.models import Sequential, load_model, Model

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, MaxPool2D
from tensorflow.keras.layers import ZeroPadding2D, BatchNormalization, AveragePooling2D, Input, Lambda
from tensorflow.keras.layers import Embedding,Conv1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Dense, Activation, MaxPooling1D


from tensorflow.keras.optimizers import SGD, Adam

from tensorflow.keras.utils import to_categorical



from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LambdaCallback


from tensorflow.keras import regularizers




import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling



In [124]:
def build_model_regression(X_train, y_train):
    model = Sequential([
    Dense(64, activation='relu', input_shape=[len(X_train.columns)]),
    Dense(64, activation='relu'),
    Dense(1)])
    model.compile(loss='mse', optimizer="adam", metrics=['mae', 'mse'])
    model.fit(X_train,y_train,epochs=10,validation_split=0.2,verbose=0)
    return model

dl_model = build_model_regression(X.copy(), y)

pred_y_dl = dl_model.predict(df_test)


In [126]:
dl_model = build_model_regression(X.copy(), y)

In [129]:
pred_y_dl = dl_model.predict(df_test)


create_submission_file(sample.copy(), pred_y_dl, "DL_1")

'DL_1 csv submission created successful'

In [ ]:
# !pip install git+https://github.com/tensorflow/docs







model = build_model_regression(X_train)
model.summary()


EPOCHS = 1000

history=model.fit(X_train,y_train,epochs=1000,validation_split=0.2,verbose=0,callbacks=[tfdocs.modeling.EpochDots()])



#graph 1
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({'Basic': history}, metric = "mae")
plt.ylim([0, 10])
plt.ylabel('MAE [MPG]') #mpg : target column name

#graph 2
plotter.plot({'Basic': history}, metric = "mse")
plt.ylim([0, 20])
plt.ylabel('MSE [MPG^2]')


model = build_model_regression()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

early_history = model.fit(normed_train_data, train_labels, 
                    epochs=EPOCHS, validation_split = 0.2, verbose=0, 
                    callbacks=[early_stop, tfdocs.modeling.EpochDots()])

#again write graph 1 and 2
plotter.plot({'Early Stopping': early_history}, metric = "mae")
plt.ylim([0, 10])
plt.ylabel('MAE [MPG]')


loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)
print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))


test_predictions = model.predict(normed_test_data).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)
